In [1]:
#Imports
import pandas as pd
import os
import tensorflow as tf
from keras import backend as K

from utils.modelgenerator import *
from utils.modelhandler import *
from utils.datahandler import *

#Get data 
cwd = os.path.normpath(os.getcwd())
df = pd.read_csv(cwd+'/data/df_with_final_features.csv', index_col='Date') #df = pd.read_csv('user5.csv')
df.index = pd.to_datetime(df.index)
#df = df[['User5', 'temp', 'rhum']]
df.fillna(0, inplace=True)

df_array = []
for idx in range(30):
    df_array.append(df[[f'User{idx+1}', 'temp', 'rhum', 'wspd', 'PC1', 'hour sin', 'hour cos', f'User{idx+1}_lag_24hrs']])

#df_array[3].head(3)

#Train, Validation and Test datasets
sequence_length = 25
batch_size = 16
num_features = df_array[0].shape[1]

dh = Datahandler()

X_train, y_train, X_val, y_val, X_test, y_test = {}, {}, {}, {}, {}, {}

#Create Train, Validation and Test datasets
for idx, df in enumerate(df_array):
    n = len(df)
    train_df = df[0:int(n*0.7)]
    val_df = df[int(n*0.7):int(n*0.9)]
    test_df = df[int(n*0.9):]

    # Min max sclaing
    train_df = dh.min_max_scaling(train_df)
    val_df = dh.min_max_scaling(val_df)
    test_df = dh.min_max_scaling(test_df)

    # Sequencing
    train_sequences = dh.create_sequences(train_df, sequence_length)
    val_sequences = dh.create_sequences(val_df, sequence_length)
    test_sequences = dh.create_sequences(test_df, sequence_length)

    #Split into feature and label
    X_train[f'user{idx+1}'], y_train[f'user{idx+1}'] = dh.prepare_data(train_sequences, batch_size)
    X_val[f'user{idx+1}'], y_val[f'user{idx+1}'] = dh.prepare_data(val_sequences, batch_size)
    X_test[f'user{idx+1}'], y_test[f'user{idx+1}'] = dh.prepare_data(test_sequences, batch_size)

#General Hyperparameters
# #All models
horizon = 1
max_epochs = 100
m1 = ModelGenerator()
mh = Modelhandler()

loss = tf.keras.losses.MeanSquaredError()
metrics=[
    tf.keras.metrics.RootMeanSquaredError(), 
    tf.keras.metrics.MeanAbsolutePercentageError(),
    tf.keras.metrics.MeanAbsoluteError(),
]

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')
timing_callback = TimingCallback()
custom_callback = CustomCallback()
#model_checkpoint = ModelCheckpoint('models/best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
callbacks=[early_stopping, timing_callback, custom_callback] #model_checkpoint


In [2]:
def avg_weights_with_noise_fedprox(weight_list, clip_threshold=None, noise_scale=0.001, proximal_term=0.1):
    avg_grad = list()

    for grad_list_tuple in zip(*weight_list):
        layer_mean = tf.math.reduce_mean(grad_list_tuple, axis=0)

        if clip_threshold is not None:
            layer_mean = tf.clip_by_value(layer_mean, -clip_threshold, clip_threshold)

        noise = tf.random.normal(shape=layer_mean.shape, mean=0.0, stddev=noise_scale)
        noisy_layer_mean = layer_mean + noise

        # Add FedProx proximal term
        proximal_update = -proximal_term * noisy_layer_mean

        avg_grad.append(noisy_layer_mean + proximal_update)

    return avg_grad

# Federated Learning benchmark Dense

In [3]:
results = pd.DataFrame(columns=['architecture', 'train_time', 'avg_time_epoch', 'mse','mse_std', 'rmse','rmse_std','mape','mape_std','mae','mae_std'])

In [4]:
num_clusters = 4
y = np.loadtxt(f'evaluations/federated_learning/clusters_KMeans{num_clusters}_dtw.csv', delimiter=',').astype(int)

cluster_users = {i: [] for i in range(num_clusters)}

# Iterate through each cluster
for cluster_number in range(num_clusters):
    users_in_cluster = np.where(y == cluster_number)[0] +1
    cluster_users[cluster_number] = users_in_cluster
cluster_users


{0: array([ 1,  2,  4,  5,  7, 10, 11, 12, 15, 16, 17, 21, 22, 23, 24, 25, 28,
        30], dtype=int64),
 1: array([29], dtype=int64),
 2: array([ 6, 13, 18, 20, 26], dtype=int64),
 3: array([ 3,  8,  9, 14, 19, 27], dtype=int64)}

### Dense Model

In [5]:
#Dense MODEL 1 ------------------------------------------------------------------
architecture = "Dense_L3_U16"

dense_units = 16
num_layers = 3

# Create global models for each cluser (6)
for cluster in range(num_clusters):
#Build and save global model
    global_model = m1.build_dense_model(X_train[f'user{1}'], horizon, num_layers,  dense_units, batch_size)
    global_model.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster}/{architecture}/FederatedRound{0}")

  
federated_rounds = 3
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    for cluster_number, users_in_cluster in cluster_users.items():
        print(f"Cluster {cluster_number}:")

        #Get global models weights
        global_model = keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_round}", compile=False)
        global_model_weights = global_model.get_weights()

        #initial list for local model weights
        local_model_weight_list = list()


        #for idx, user in enumerate(df_array): 
        for user_index in users_in_cluster:
            user_df = df_array[user_index-1]  # Get the user's DataFrame from the array
            print(f"User {user_index}") 
                      
            #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
            local_model = m1.build_dense_model(X_train[f'user{user_index}'], horizon, num_layers,  dense_units, batch_size)
            local_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006), metrics=metrics)

            #set local model weight to the weight of the global model
            local_model.set_weights(global_model_weights)
            
            #Fit local model to local data
            histroy, user_results = mh.compile_fit_evaluate_model(
                model=local_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = max_epochs, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025),
            )
            #add model weights to list        
            local_model_weights = local_model.get_weights()
            local_model_weight_list.append(local_model_weights)
        
            #clear session to free memory after each communication round
            K.clear_session()
        
        #to get the average over all the local model, we simply take the sum of the scaled weights
        average_weights = avg_weights_with_noise_fedprox(local_model_weight_list)
        #update global model 
        global_model.set_weights(average_weights)
        #Save global models
        global_model.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_round+1}")
        print("Saved Global models")


#Evaluation
all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

for cluster_number, users_in_cluster in cluster_users.items():
    print(f"Cluster {cluster_number}:")

    #Get global models weights
    global_model = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_rounds}", compile=False)

    #for idx, user in enumerate(df_array): 
    for user_index in users_in_cluster:
        print("User: ", user_index)
        for round in range(3):
            global_model = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_rounds}", compile=False)
            local_model = m1.build_dense_model(X_train[f'user{user_index}'], horizon, num_layers,  dense_units, batch_size)
            local_model.set_weights(global_model.get_weights())
            
            histroy, user_results = mh.compile_fit_evaluate_model(
                model=local_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = 1, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025)
            )
            # Add the 'architecture' column from dense_user_results to dense_results
            all_results = pd.merge(all_results, user_results, how='outer')  

for idx in range(len(df_array)):
    new_row = {
        'architecture': architecture,
        'train_time': all_results[all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : all_results[all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': all_results[all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : all_results[all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': all_results[all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : all_results[all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': all_results[all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : all_results[all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': all_results[all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : all_results[all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    results.loc[len(results)] = new_row

results

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/Dense_L3_U16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/Dense_L3_U16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/Dense_L3_U16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/Dense_L3_U16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_2/Dense_L3_U16/FederatedRound0\assets


...

Saved Global models
Cluster 3:
User 3
User 8
User 9
User 14
User 19
User 27


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/Dense_L3_U16/FederatedRound3\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/Dense_L3_U16/FederatedRound3\assets


Saved Global models
Cluster 0:
User:  1
User:  2
User:  4
User:  5
User:  7
User:  10
User:  11
User:  12
User:  15
User:  16
User:  17
User:  21
User:  22
User:  23
User:  24
User:  25
User:  28
User:  30
Cluster 1:
User:  29
Cluster 2:
User:  6
User:  13
User:  18
User:  20
User:  26
Cluster 3:
User:  3
User:  8
User:  9
User:  14
User:  19
User:  27


,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_L3_U16,0.940380,0.861906,0.034988,0.001341,0.187027,0.003599,1.253973e+05,46866.889580,0.134483,0.008541
1,Dense_L3_U16,0.915245,0.844485,0.018157,0.000662,0.134733,0.002444,1.812737e+05,40845.543470,0.078444,0.002914
2,Dense_L3_U16,0.912691,0.841930,0.019733,0.000712,0.140459,0.002547,6.088697e+04,15466.857347,0.081351,0.006668
3,Dense_L3_U16,0.920840,0.847734,0.024768,0.002448,0.157252,0.007675,1.362346e+05,11675.268054,0.065159,0.000484
4,Dense_L3_U16,0.920456,0.846996,0.013719,0.000217,0.117126,0.000926,1.066967e+05,20105.360464,0.075844,0.005128
5,Dense_L3_U16,0.919330,0.847568,0.020080,0.000217,0.141703,0.000765,1.373506e+05,10953.872552,0.106075,0.004135
6,Dense_L3_U16,0.916010,0.842238,0.019962,0.000498,0.141279,0.001763,8.046139e+04,6548.076281,0.081299,0.001126
7,Dense_L3_U16,1.049577,0.980151,0.027332,0.000426,0.165320,0.001289,6.328486e+04,24972.401719,0.108425,0.002218
8,Dense_L3_U16,0.917170,0.844064,0.031483,0.003176,0.177281,0.009056,7.085241e+01,5.992206,0.105285,0.003564
9,Dense_L3_U16,0.917840,0.846082,0.013314,0.000794,0.115353,0.003414,2.228576e+06,284074.086775,0.067837,0.003765


In [6]:
results.to_csv(f'evaluations/clustered_federated_learning/{architecture}_2.csv')
print(results["mse"].mean())
results

0.018702405634232692


,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_L3_U16,0.940380,0.861906,0.034988,0.001341,0.187027,0.003599,1.253973e+05,46866.889580,0.134483,0.008541
1,Dense_L3_U16,0.915245,0.844485,0.018157,0.000662,0.134733,0.002444,1.812737e+05,40845.543470,0.078444,0.002914
2,Dense_L3_U16,0.912691,0.841930,0.019733,0.000712,0.140459,0.002547,6.088697e+04,15466.857347,0.081351,0.006668
3,Dense_L3_U16,0.920840,0.847734,0.024768,0.002448,0.157252,0.007675,1.362346e+05,11675.268054,0.065159,0.000484
4,Dense_L3_U16,0.920456,0.846996,0.013719,0.000217,0.117126,0.000926,1.066967e+05,20105.360464,0.075844,0.005128
5,Dense_L3_U16,0.919330,0.847568,0.020080,0.000217,0.141703,0.000765,1.373506e+05,10953.872552,0.106075,0.004135
6,Dense_L3_U16,0.916010,0.842238,0.019962,0.000498,0.141279,0.001763,8.046139e+04,6548.076281,0.081299,0.001126
7,Dense_L3_U16,1.049577,0.980151,0.027332,0.000426,0.165320,0.001289,6.328486e+04,24972.401719,0.108425,0.002218
8,Dense_L3_U16,0.917170,0.844064,0.031483,0.003176,0.177281,0.009056,7.085241e+01,5.992206,0.105285,0.003564
9,Dense_L3_U16,0.917840,0.846082,0.013314,0.000794,0.115353,0.003414,2.228576e+06,284074.086775,0.067837,0.003765


### BiLSTM

In [7]:
architecture = "BiLSTM_L2_U20"

lstm_layers = 2
lstm_units = 20

# Create global models for each cluser (6)
for cluster in range(num_clusters):
#Build and save global model
    global_model = m1.build_bilstm_model(X_train[f'user{1}'], horizon, num_layers=lstm_layers, units=lstm_units, batch_size=batch_size)
    global_model.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster}/{architecture}/FederatedRound{0}")

  
federated_rounds = 3
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    for cluster_number, users_in_cluster in cluster_users.items():
        print(f"Cluster {cluster_number}:")

        #Get global models weights
        global_model = keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_round}", compile=False)
        global_model_weights = global_model.get_weights()

        #initial list for local model weights
        local_model_weight_list = list()


        #for idx, user in enumerate(df_array): 
        for user_index in users_in_cluster:
            user_df = df_array[user_index-1]  # Get the user's DataFrame from the array
            print(f"User {user_index}") 
                      
            #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
            local_model = m1.build_bilstm_model(X_train[f'user{user_index}'], horizon, num_layers=lstm_layers, units=lstm_units, batch_size=batch_size)
            local_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006), metrics=metrics)

            #set local model weight to the weight of the global model
            local_model.set_weights(global_model_weights)
            
            #Fit local model to local data
            histroy, user_results = mh.compile_fit_evaluate_model(
                model=local_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = max_epochs, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006),
            )
            #add model weights to list        
            local_model_weights = local_model.get_weights()
            local_model_weight_list.append(local_model_weights)
        
            #clear session to free memory after each communication round
            K.clear_session()
        
        #to get the average over all the local model, we simply take the sum of the scaled weights
        average_weights = avg_weights_with_noise_fedprox(local_model_weight_list)
        #update global model 
        global_model.set_weights(average_weights)
        #Save global models
        global_model.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_round+1}")
        print("Saved Global models")


#Evaluation
all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

for cluster_number, users_in_cluster in cluster_users.items():
    print(f"Cluster {cluster_number}:")

    #Get global models weights
    global_model = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_rounds}", compile=False)

    #for idx, user in enumerate(df_array): 
    for user_index in users_in_cluster:
        print("User: ", user_index)
        for round in range(3):
            global_model = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_rounds}", compile=False)
            local_model = m1.build_bilstm_model(X_train[f'user{user_index}'], horizon, num_layers=lstm_layers, units=lstm_units, batch_size=batch_size)
            local_model.set_weights(global_model.get_weights())
            
            histroy, user_results = mh.compile_fit_evaluate_model(
                model=local_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = 1, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006)
            )
            # Add the 'architecture' column from dense_user_results to dense_results
            all_results = pd.merge(all_results, user_results, how='outer')  

for idx in range(len(df_array)):
    new_row = {
        'architecture': architecture,
        'train_time': all_results[all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : all_results[all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': all_results[all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : all_results[all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': all_results[all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : all_results[all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': all_results[all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : all_results[all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': all_results[all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : all_results[all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    results.loc[len(results)] = new_row

results

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/BiLSTM_L2_U20/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/BiLSTM_L2_U20/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/BiLSTM_L2_U20/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/BiLSTM_L2_U20/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_2/BiLSTM_L2_U20/FederatedRound0\assets


...

Saved Global models
Cluster 3:
User 3
User 8
User 9
User 14
User 19
User 27


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/BiLSTM_L2_U20/FederatedRound3\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/BiLSTM_L2_U20/FederatedRound3\assets


Saved Global models
Cluster 0:
User:  1
User:  2
User:  4
User:  5
User:  7
User:  10
User:  11
User:  12
User:  15
User:  16
User:  17
User:  21
User:  22
User:  23
User:  24
User:  25
User:  28
User:  30
Cluster 1:
User:  29
Cluster 2:
User:  6
User:  13
User:  18
User:  20
User:  26
Cluster 3:
User:  3
User:  8
User:  9
User:  14
User:  19
User:  27


,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_L3_U16,0.935475,0.859926,0.034367,0.000558,0.185379,0.001503,8.435151e+04,5276.563581,0.134481,0.001163
1,Dense_L3_U16,1.062256,0.988570,0.016742,0.000258,0.129387,0.000998,2.894397e+05,17633.278160,0.076471,0.001340
2,Dense_L3_U16,0.926651,0.854965,0.018892,0.000258,0.137447,0.000939,4.547944e+04,6384.431282,0.076337,0.000670
3,Dense_L3_U16,0.923367,0.851295,0.022242,0.000605,0.149128,0.002022,1.328210e+05,7263.428431,0.063521,0.000920
4,Dense_L3_U16,0.921280,0.849261,0.013165,0.000217,0.114735,0.000947,8.575916e+04,7781.146682,0.073596,0.001301
5,Dense_L3_U16,0.922215,0.848528,0.019220,0.000138,0.138637,0.000497,1.000368e+05,8598.919788,0.100257,0.002861
6,Dense_L3_U16,0.933126,0.859773,0.018804,0.000615,0.137116,0.002232,7.152170e+04,6090.935211,0.081467,0.001777
7,Dense_L3_U16,0.944919,0.869899,0.023224,0.000673,0.152384,0.002216,1.423177e+04,6381.654232,0.098019,0.000635
8,Dense_L3_U16,0.924121,0.850769,0.024089,0.001217,0.155172,0.003905,6.658551e+01,8.269819,0.090012,0.001159
9,Dense_L3_U16,1.082936,1.008213,0.012787,0.000257,0.113077,0.001139,2.638208e+06,370528.215327,0.069678,0.004041


In [8]:
results.to_csv(f'evaluations/clustered_federated_learning/{architecture}.csv')
results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_L3_U16,0.935475,0.859926,0.034367,0.000558,0.185379,0.001503,8.435151e+04,5276.563581,0.134481,0.001163
1,Dense_L3_U16,1.062256,0.988570,0.016742,0.000258,0.129387,0.000998,2.894397e+05,17633.278160,0.076471,0.001340
2,Dense_L3_U16,0.926651,0.854965,0.018892,0.000258,0.137447,0.000939,4.547944e+04,6384.431282,0.076337,0.000670
3,Dense_L3_U16,0.923367,0.851295,0.022242,0.000605,0.149128,0.002022,1.328210e+05,7263.428431,0.063521,0.000920
4,Dense_L3_U16,0.921280,0.849261,0.013165,0.000217,0.114735,0.000947,8.575916e+04,7781.146682,0.073596,0.001301
5,Dense_L3_U16,0.922215,0.848528,0.019220,0.000138,0.138637,0.000497,1.000368e+05,8598.919788,0.100257,0.002861
6,Dense_L3_U16,0.933126,0.859773,0.018804,0.000615,0.137116,0.002232,7.152170e+04,6090.935211,0.081467,0.001777
7,Dense_L3_U16,0.944919,0.869899,0.023224,0.000673,0.152384,0.002216,1.423177e+04,6381.654232,0.098019,0.000635
8,Dense_L3_U16,0.924121,0.850769,0.024089,0.001217,0.155172,0.003905,6.658551e+01,8.269819,0.090012,0.001159
9,Dense_L3_U16,1.082936,1.008213,0.012787,0.000257,0.113077,0.001139,2.638208e+06,370528.215327,0.069678,0.004041


### CNN

In [9]:
architecture = "CNN_L4_f8k1_d16"
num_layers = 4
filter_size = 8
kernel_size = 1
dense_units = 16

# Create global models for each cluser (6)
for cluster in range(num_clusters):
#Build and save global model
    global_model = m1.build_cnn_model(X_train[f'user{1}'], horizon, num_layers=num_layers, filter=filter_size, kernel_size=kernel_size, dense_units=dense_units, batch_size=batch_size)
    global_model.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster}/{architecture}/FederatedRound{0}")

  
federated_rounds = 3
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    for cluster_number, users_in_cluster in cluster_users.items():
        print(f"Cluster {cluster_number}:")

        #Get global models weights
        global_model = keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_round}", compile=False)
        global_model_weights = global_model.get_weights()

        #initial list for local model weights
        local_model_weight_list = list()


        #for idx, user in enumerate(df_array): 
        for user_index in users_in_cluster:
            user_df = df_array[user_index-1]  # Get the user's DataFrame from the array
            print(f"User {user_index}") 
                      
            #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
            local_model = m1.build_cnn_model(X_train[f'user{user_index}'], horizon, num_layers=num_layers, filter=filter_size, kernel_size=kernel_size, dense_units=dense_units, batch_size=batch_size)
            local_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006), metrics=metrics)

            #set local model weight to the weight of the global model
            local_model.set_weights(global_model_weights)
            
            #Fit local model to local data
            histroy, user_results = mh.compile_fit_evaluate_model(
                model=local_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = max_epochs, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006),
            )
            #add model weights to list        
            local_model_weights = local_model.get_weights()
            local_model_weight_list.append(local_model_weights)
        
            #clear session to free memory after each communication round
            K.clear_session()
        
        #to get the average over all the local model, we simply take the sum of the scaled weights
        average_weights = avg_weights_with_noise_fedprox(local_model_weight_list)
        #update global model 
        global_model.set_weights(average_weights)
        #Save global models
        global_model.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_round+1}")
        print("Saved Global models")


#Evaluation
all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

for cluster_number, users_in_cluster in cluster_users.items():
    print(f"Cluster {cluster_number}:")

    #Get global models weights
    global_model = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_rounds}", compile=False)

    #for idx, user in enumerate(df_array): 
    for user_index in users_in_cluster:
        print("User: ", user_index)
        for round in range(3):
            global_model = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_rounds}", compile=False)
            local_model = m1.build_cnn_model(X_train[f'user{user_index}'], horizon, num_layers=num_layers, filter=filter_size, kernel_size=kernel_size, dense_units=dense_units, batch_size=batch_size)
            local_model.set_weights(global_model.get_weights())
            
            histroy, user_results = mh.compile_fit_evaluate_model(
                model=local_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = 1, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006)
            )
            # Add the 'architecture' column from dense_user_results to dense_results
            all_results = pd.merge(all_results, user_results, how='outer')  

for idx in range(len(df_array)):
    new_row = {
        'architecture': architecture,
        'train_time': all_results[all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : all_results[all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': all_results[all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : all_results[all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': all_results[all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : all_results[all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': all_results[all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : all_results[all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': all_results[all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : all_results[all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    results.loc[len(results)] = new_row

results

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/CNN_L4_f8k1_d16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/CNN_L4_f8k1_d16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/CNN_L4_f8k1_d16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/CNN_L4_f8k1_d16/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_2/CNN_L4_f8k1_d16/FederatedRound0\assets


...

Saved Global models
Cluster 3:
User 3
User 8
User 9
User 14
User 19
User 27


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/CNN_L4_f8k1_d16/FederatedRound3\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/CNN_L4_f8k1_d16/FederatedRound3\assets


Saved Global models
Cluster 0:
User:  1
User:  2
User:  4
User:  5
User:  7
User:  10
User:  11
User:  12
User:  15
User:  16
User:  17
User:  21
User:  22
User:  23
User:  24
User:  25
User:  28
User:  30
Cluster 1:
User:  29
Cluster 2:
User:  6
User:  13
User:  18
User:  20
User:  26
Cluster 3:
User:  3
User:  8
User:  9
User:  14
User:  19
User:  27


,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_L3_U16,0.935475,0.859926,0.034367,0.000558,0.185379,0.001503,8.435151e+04,5276.563581,0.134481,0.001163
1,Dense_L3_U16,1.062256,0.988570,0.016742,0.000258,0.129387,0.000998,2.894397e+05,17633.278160,0.076471,0.001340
2,Dense_L3_U16,0.926651,0.854965,0.018892,0.000258,0.137447,0.000939,4.547944e+04,6384.431282,0.076337,0.000670
3,Dense_L3_U16,0.923367,0.851295,0.022242,0.000605,0.149128,0.002022,1.328210e+05,7263.428431,0.063521,0.000920
4,Dense_L3_U16,0.921280,0.849261,0.013165,0.000217,0.114735,0.000947,8.575916e+04,7781.146682,0.073596,0.001301
...,...,...,...,...,...,...,...,...,...,...,...
85,CNN_L4_f8k1_d16,1.277114,1.194817,0.064023,0.002059,0.253007,0.004058,1.367990e+05,7454.326952,0.197688,0.003892
86,CNN_L4_f8k1_d16,1.331510,1.254284,0.031042,0.000349,0.176185,0.000993,1.511879e+05,6208.980547,0.136247,0.000422
87,CNN_L4_f8k1_d16,1.293389,1.204516,0.014139,0.000117,0.118905,0.000493,9.032578e+04,3271.124674,0.055589,0.000773
88,CNN_L4_f8k1_d16,1.281466,1.204957,0.032630,0.000636,0.180633,0.001755,1.389982e+06,76479.546169,0.143277,0.000450


In [10]:
results.to_csv(f'evaluations/clustered_federated_learning/{architecture}.csv')
results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Dense_L3_U16,0.935475,0.859926,0.034367,0.000558,0.185379,0.001503,8.435151e+04,5276.563581,0.134481,0.001163
1,Dense_L3_U16,1.062256,0.988570,0.016742,0.000258,0.129387,0.000998,2.894397e+05,17633.278160,0.076471,0.001340
2,Dense_L3_U16,0.926651,0.854965,0.018892,0.000258,0.137447,0.000939,4.547944e+04,6384.431282,0.076337,0.000670
3,Dense_L3_U16,0.923367,0.851295,0.022242,0.000605,0.149128,0.002022,1.328210e+05,7263.428431,0.063521,0.000920
4,Dense_L3_U16,0.921280,0.849261,0.013165,0.000217,0.114735,0.000947,8.575916e+04,7781.146682,0.073596,0.001301
...,...,...,...,...,...,...,...,...,...,...,...
85,CNN_L4_f8k1_d16,1.277114,1.194817,0.064023,0.002059,0.253007,0.004058,1.367990e+05,7454.326952,0.197688,0.003892
86,CNN_L4_f8k1_d16,1.331510,1.254284,0.031042,0.000349,0.176185,0.000993,1.511879e+05,6208.980547,0.136247,0.000422
87,CNN_L4_f8k1_d16,1.293389,1.204516,0.014139,0.000117,0.118905,0.000493,9.032578e+04,3271.124674,0.055589,0.000773
88,CNN_L4_f8k1_d16,1.281466,1.204957,0.032630,0.000636,0.180633,0.001755,1.389982e+06,76479.546169,0.143277,0.000450


### Transformer

In [5]:
architecture = "Transformer_ED2_h4_d32"
num_layers = 2
num_heads = 4
dense_units = 32

# Create global models for each cluser (6)
for cluster in range(num_clusters):
#Build and save global model
    global_model = m1.build_transformer_model(X_train[f'user{1}'], horizon, batch_size, sequence_length, num_layers, num_features, num_heads, dense_units, m1)
    global_model.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster}/{architecture}/FederatedRound{0}")

  
federated_rounds = 3
for federated_round  in range(federated_rounds):
    print("Started Federated training round ----------", federated_round+1, f"/ {federated_rounds}")

    for cluster_number, users_in_cluster in cluster_users.items():
        print(f"Cluster {cluster_number}:")

        #Get global models weights
        global_model = keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_round}", compile=False)
        global_model_weights = global_model.get_weights()

        #initial list for local model weights
        local_model_weight_list = list()


        #for idx, user in enumerate(df_array): 
        for user_index in users_in_cluster:
            user_df = df_array[user_index-1]  # Get the user's DataFrame from the array
            print(f"User {user_index}") 
                      
            #build and compile local model X_train, batch_size, horizon, dense_units,  expert_units, num_experts, m1
            local_model = m1.build_transformer_model(X_train[f'user{user_index}'], horizon, batch_size, sequence_length, num_layers, num_features, num_heads, dense_units, m1)
            local_model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006), metrics=metrics)

            #set local model weight to the weight of the global model
            local_model.set_weights(global_model_weights)
            
            #Fit local model to local data
            histroy, user_results = mh.compile_fit_evaluate_model(
                model=local_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = max_epochs, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006),
            )
            #add model weights to list        
            local_model_weights = local_model.get_weights()
            local_model_weight_list.append(local_model_weights)
        
            #clear session to free memory after each communication round
            K.clear_session()
        
        #to get the average over all the local model, we simply take the sum of the scaled weights
        average_weights = avg_weights_with_noise_fedprox(local_model_weight_list)
        #update global model 
        global_model.set_weights(average_weights)
        #Save global models
        global_model.save(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_round+1}")
        print("Saved Global models")


#Evaluation
all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])

for cluster_number, users_in_cluster in cluster_users.items():
    print(f"Cluster {cluster_number}:")

    #Get global models weights
    global_model = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_rounds}", compile=False)

    #for idx, user in enumerate(df_array): 
    for user_index in users_in_cluster:
        print("User: ", user_index)
        for round in range(3):
            global_model = tf.keras.models.load_model(cwd + f"/models/FL/Dense/global_dense_model/cluster_{cluster_number}/{architecture}/FederatedRound{federated_rounds}", compile=False)
            local_model = m1.build_transformer_model(X_train[f'user{user_index}'], horizon, batch_size, sequence_length, num_layers, num_features, num_heads, dense_units, m1)
            local_model.set_weights(global_model.get_weights())
            
            histroy, user_results = mh.compile_fit_evaluate_model(
                model=local_model, 
                loss=loss, 
                metrics=metrics, 
                X_train=X_train[f'user{user_index}'],
                y_train = y_train[f'user{user_index}'], 
                max_epochs = 1, 
                batch_size=batch_size, 
                X_val=X_val[f'user{user_index}'], 
                y_val=y_val[f'user{user_index}'], 
                X_test=X_test[f'user{user_index}'], 
                y_test=y_test[f'user{user_index}'], 
                callbacks=callbacks, 
                user=f'user{user_index}', 
                hyper=architecture,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006)
            )
            # Add the 'architecture' column from dense_user_results to dense_results
            all_results = pd.merge(all_results, user_results, how='outer')  

for idx in range(len(df_array)):
    new_row = {
        'architecture': architecture,
        'train_time': all_results[all_results["user"]==f"user{idx+1}"]["train_time"].mean(), 
        'avg_time_epoch' : all_results[all_results["user"]==f"user{idx+1}"]["avg_time_epoch"].mean(),
        'mse': all_results[all_results["user"]==f"user{idx+1}"]["mse"].mean(),
        'mse_std' : all_results[all_results["user"]==f"user{idx+1}"]["mse"].std(),
        'rmse': all_results[all_results["user"]==f"user{idx+1}"]["rmse"].mean(),
        'rmse_std' : all_results[all_results["user"]==f"user{idx+1}"]["rmse"].std(),
        'mape': all_results[all_results["user"]==f"user{idx+1}"]["mape"].mean(),
        'mape_std' : all_results[all_results["user"]==f"user{idx+1}"]["mape"].std(),
        'mae': all_results[all_results["user"]==f"user{idx+1}"]["mae"].mean(),
        'mae_std' : all_results[all_results["user"]==f"user{idx+1}"]["mae"].std(),
    }
    results.loc[len(results)] = new_row

results

INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/Transformer_ED2_h4_d32/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_0/Transformer_ED2_h4_d32/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/Transformer_ED2_h4_d32/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_1/Transformer_ED2_h4_d32/FederatedRound0\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_2/Transformer_ED2_h4_d32/FederatedRound0\assets


...

Saved Global models
Cluster 3:
User 3
User 8
User 9
User 14
User 19
User 27


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/Transformer_ED2_h4_d32/FederatedRound3\assets


INFO:tensorflow:Assets written to: c:\Users\rs1044\bwSyncShare\02Python Code\MoE-based-FL-for-secure-STLF/models/FL/Dense/global_dense_model/cluster_3/Transformer_ED2_h4_d32/FederatedRound3\assets


Saved Global models
Cluster 0:
User:  1
User:  2
User:  4
User:  5
User:  7
User:  10
User:  11
User:  12
User:  15
User:  16
User:  17
User:  21
User:  22
User:  23
User:  24
User:  25
User:  28
User:  30
Cluster 1:
User:  29
Cluster 2:
User:  6
User:  13
User:  18
User:  20
User:  26
Cluster 3:
User:  3
User:  8
User:  9
User:  14
User:  19
User:  27


,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Transformer_ED2_h4_d32,7.673781,7.430955,0.035487,0.000148,0.188381,0.000393,1.537773e+05,22941.154769,0.133593,0.003287
1,Transformer_ED2_h4_d32,7.854902,7.608189,0.024108,0.000500,0.155263,0.001610,2.429149e+05,22201.225530,0.105893,0.006255
2,Transformer_ED2_h4_d32,8.347739,8.063070,0.026229,0.000406,0.161952,0.001252,7.451079e+04,17137.503589,0.088409,0.004833
3,Transformer_ED2_h4_d32,8.010622,7.743855,0.030692,0.000381,0.175189,0.001089,1.499930e+05,24352.994086,0.077027,0.004299
4,Transformer_ED2_h4_d32,7.947295,7.684051,0.022212,0.000209,0.149035,0.000699,1.259777e+05,2678.122143,0.104704,0.001738
5,Transformer_ED2_h4_d32,8.206718,7.931443,0.030179,0.000641,0.173715,0.001847,1.971175e+05,19641.948260,0.139380,0.003799
6,Transformer_ED2_h4_d32,7.725709,7.477860,0.033806,0.000575,0.183859,0.001562,9.797814e+04,14706.295308,0.116262,0.006279
7,Transformer_ED2_h4_d32,8.718048,8.433021,0.053375,0.000430,0.231029,0.000930,1.050932e+05,53885.437799,0.171563,0.006641
8,Transformer_ED2_h4_d32,8.526953,8.250996,0.033184,0.001535,0.182133,0.004234,9.468775e+01,47.398622,0.112293,0.009783
9,Transformer_ED2_h4_d32,7.600033,7.353819,0.028645,0.000812,0.169238,0.002396,3.486233e+06,591496.867432,0.118109,0.008125


In [6]:
results.to_csv(f'evaluations/clustered_federated_learning/{architecture}.csv')
results

,architecture,train_time,avg_time_epoch,mse,mse_std,rmse,rmse_std,mape,mape_std,mae,mae_std
0,Transformer_ED2_h4_d32,7.673781,7.430955,0.035487,0.000148,0.188381,0.000393,1.537773e+05,22941.154769,0.133593,0.003287
1,Transformer_ED2_h4_d32,7.854902,7.608189,0.024108,0.000500,0.155263,0.001610,2.429149e+05,22201.225530,0.105893,0.006255
2,Transformer_ED2_h4_d32,8.347739,8.063070,0.026229,0.000406,0.161952,0.001252,7.451079e+04,17137.503589,0.088409,0.004833
3,Transformer_ED2_h4_d32,8.010622,7.743855,0.030692,0.000381,0.175189,0.001089,1.499930e+05,24352.994086,0.077027,0.004299
4,Transformer_ED2_h4_d32,7.947295,7.684051,0.022212,0.000209,0.149035,0.000699,1.259777e+05,2678.122143,0.104704,0.001738
5,Transformer_ED2_h4_d32,8.206718,7.931443,0.030179,0.000641,0.173715,0.001847,1.971175e+05,19641.948260,0.139380,0.003799
6,Transformer_ED2_h4_d32,7.725709,7.477860,0.033806,0.000575,0.183859,0.001562,9.797814e+04,14706.295308,0.116262,0.006279
7,Transformer_ED2_h4_d32,8.718048,8.433021,0.053375,0.000430,0.231029,0.000930,1.050932e+05,53885.437799,0.171563,0.006641
8,Transformer_ED2_h4_d32,8.526953,8.250996,0.033184,0.001535,0.182133,0.004234,9.468775e+01,47.398622,0.112293,0.009783
9,Transformer_ED2_h4_d32,7.600033,7.353819,0.028645,0.000812,0.169238,0.002396,3.486233e+06,591496.867432,0.118109,0.008125


### Ensemble

In [ ]:
#For each of the 3 user
for idx in range(len(df_array)):
    print("User: ", idx+1)
    for round in range(3):

        num_base_models = 4
        # Train base models
        base_models = []

        start_time = time.time()
        for _ in range(num_base_models):
            model = m1.build_bilstm_model(X_train[f'user{idx+1}'], horizon, num_layers=2, units=20, batch_size=batch_size)
            
            model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),metrics=metrics)

            model.fit(X_train[f'user{idx+1}'], y_train[f'user{idx+1}'], epochs=25, batch_size=batch_size, verbose=0)
            base_models.append(model)

        # Make predictions with base models
        predictions = np.mean([model.predict(X_test[f'user{idx+1}'], batch_size=batch_size, verbose=0) for model in base_models], axis=0)

                   
        end_time = time.time()
        train_time = end_time - start_time
        avg_time_epoch = train_time/200

        user_results = mh.evaluate_ensemble(y_test[f'user{idx+1}'], predictions, user=f'user{idx+1}', hyper="ensemble_bilstm_avg", train_time=train_time, avg_time_epoch=avg_time_epoch)
        all_results = pd.merge(all_results, user_results, how='outer')

In [ ]:
# Ensemble Dense - Averaging


In [ ]:
results.to_csv(f'evaluations/clustered_federated_learning/ensemble_dense_stacking.csv')
results